In [2]:
import pandas as pd
import numpy as np


In [45]:
def preprocces(file_path = "simplified.xlsx"):
    d = pd.read_excel(file_path)
    weight_info = d.iloc[1,:]
    d = d.drop([0,1,2], axis=0)

    for temp_name, ID in zip(["Unnamed: 0", "Unnamed: 1"], ["ID Danish", "ID English"]):
        names = d[temp_name].str.split(",")
        names = [name[0].lower() for name in names]
        d[temp_name] = names
        d = d.rename(columns = {temp_name: ID})

    d = d.drop_duplicates(subset='ID Danish', keep="first")

    #set to danish names
    d.index = d["ID Danish"]
    # tranpose such that we can use the names of the items as keys
    d = d.T

    return d

In [46]:
d = preprocces()
d.head(10)

ID Danish,jordbær,æble,banan,kartoffel,hindbær,sødmælk,persille,pære,rabarber,solbær,...,pizza topping (revet ost,skinkeost 30+ (smelteost med skinke),rejeost,gedeost,skæreost,pålæg i skiver,seitan,hakket,hakket bolle,stykker
ID Danish,jordbær,æble,banan,kartoffel,hindbær,sødmælk,persille,pære,rabarber,solbær,...,pizza topping (revet ost,skinkeost 30+ (smelteost med skinke),rejeost,gedeost,skæreost,pålæg i skiver,seitan,hakket,hakket bolle,stykker
ID English,strawberry,apple,banana,potato,raspberry,milk,parsley,pear,rhubarb,currant,...,pizza topping (grated cheese,skinkeost 30+ (processed cheese with ham),rejeost,goat cheese,cheese,cold cuts,seitan,mince,mince balls,pieces
→ParameterNavn,1,2,3,4,5,6,8,9,10,11,...,1768,1769,1770,1775,1778,1783,1784,1785,1787,1788
Energi (kJ),161.95359,232.54783,396.194566,326.336428,214.589744,264.187283,200.795833,207.024427,107.8,314.235238,...,1312.095,898.6,634.63,1322.51,734.115,938.665,818.2,404.163333,654.105,505.245
Energi (kcal),38.457949,55.064734,93.475466,77.039895,51.40641,63.171896,48.066667,49.215523,25.9,75.009762,...,315.39,216.2,151.31,318.953333,174.605,222.82,194.876944,96.593333,157.29,120.643333
Protein,0.659856,0.269358,1.140625,2.02689,1.369231,3.421768,3.7125,0.306944,0.9,1.52,...,19.34375,14.4375,17.3125,17.458333,28.28125,31.34375,28.367188,14.104167,15.375,20.385417
Protein fra Aminosyrer,0.473128,0.189219,0.738623,1.530354,NaN,3.082914,2.716532,0.236284,0.445441,NaN,...,NaN,NaN,NaN,NaN,NaN,25.536718,24.298297,10.757735,12.827748,16.981987
"Protein, deklaration",0.659856,0.269358,1.140625,2.02689,1.369231,3.352045,3.7125,0.306944,0.9,1.52,...,19.34375,14.4375,17.3125,17.458333,28.28125,31.34375,28.367188,14.104167,15.375,20.385417
Kulhydrat difference,8.347324,14.261256,22.587819,17.298029,10.644872,4.60471,7.766667,14.352651,6.8,17.15119,...,8.54125,3.1625,5.3275,2.805,1.04875,17.10625,5.310313,7.6525,8.75,7.639583
Tilgængelig kulhydrat,6.861939,12.052395,20.955092,15.859049,6.244872,4.60471,3.666667,11.131222,3,11.35119,...,8.54125,3.1625,5.3275,2.805,1.04875,11.59125,3.162535,1.185833,1.12,-0.46375


In [47]:
def get_macro_dist(receipt):
    energy_sources = ["Protein", "Fedt", "Tilgængelig kulhydrat", "Kostfibre", "Alkohol"]
    macro_dist = {name: 0 for name in energy_sources}
    for item, weight in receipt.items():
        for source in energy_sources:
            macro_dist[source] += d[item][source] * weight * 10

    return macro_dist

In [51]:
# dictionary with  {"name of item": weight in kilos}
test_receipt = {"banan": 1,
                "pasta": 0.5,
                "tomat": 0.5,
                #"fløde 50%": 0.4,
                "sødmælk": 1,
                "havregryn": 1
                }

macro_dist = get_macro_dist(test_receipt)
macro_dist

{'Protein': 239.98925756132792,
 'Fedt': 115.33431066952396,
 'Tilgængelig kulhydrat': 1217.200540652867,
 'Kostfibre': 138.32497556207235,
 'Alkohol': 0.0}

In [1]:
def get_total_energy(macro_dist):
    energy_sources =  ["Protein", "Fedt", "Tilgængelig kulhydrat", "Kostfibre", "Alkohol"]
    energy_vals = [4,9,4,2,7]
    energy = {source: val for source, val in zip(energy_sources, energy_vals)}
    total_energy = 0
    for source, _ in macro_dist.items():
        total_energy += energy[source] * macro_dist[source]

    return total_energy

total_energy = get_total_energy(macro_dist)
print("total energy in kcal:", total_energy)

NameError: name 'macro_dist' is not defined